<a href="https://colab.research.google.com/github/ItsFreakinDay/hybridCompSys/blob/task2/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%writefile task2.cu
#include <torch/extension.h>

__global__ void d_divide(int *a, int scalar, int *c, int n) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] / scalar;
    }
}

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)
#define CHECK_SIZE(x, y) TORCH_CHECK(x.is_same_size(y), #y " must be the same size as " #x)

const int block_size = 128;

__forceinline__ int calc_grid_size(int m) {
    return (m + block_size - 1) / block_size;
}

torch::Tensor divide(torch::Tensor a, int scalar) {
    CHECK_INPUT(a);

    auto c = torch::empty_like(a);
    int n = a.numel();

    d_divide<<<calc_grid_size(n), block_size>>>(
        a.data_ptr<int>(), // Изменено на int
        scalar,
        c.data_ptr<int>(), // Изменено на int
        n
    );

    return c;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("my_divide", &divide, "Custom integer division");
}


Overwriting task2.cu


In [6]:
!nvcc -o qwert task2.cu

task2.cu:1:10: fatal error: torch/extension.h: No such file or directory
    1 | #include <torch/extension.h>
      |          ^~~~~~~~~~~~~~~~~~~
compilation terminated.


In [17]:

!cd .git

/bin/bash: line 1: cd: .git: No such file or directory
